## Motivation

In this notebook, we exame the idea declared in section 1.5.5 on MNIST data, a classification task.

In [1]:
import numpy as np
import tensorflow as tf
from keras.losses import MSE
from sklearn.metrics import accuracy_score

from utils import get_gradient_loss_fn

tf.random.set_seed(42)

2024-03-21 12:32:35.087061: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## The MNIST Data

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32').reshape([-1, 28*28]) / 255.0
x_test = x_test.astype('float32').reshape([-1, 28*28]) / 255.0
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

## Train a Model with Gradient Loss

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, 'relu'),
    tf.keras.layers.Dense(128, 'relu'),
    tf.keras.layers.Dense(10, 'softmax')
])

gradient_loss_fn = get_gradient_loss_fn(
    lambda inputs: MSE(inputs[1], model(inputs[0]))
)

In [4]:
optimizer = tf.optimizers.Adam()

@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        loss = gradient_loss_fn((x, y))
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(zip(grads, model.variables))
    return loss

In [5]:
def evaluate(model):
    return accuracy_score(y_test, tf.argmax(model(x_test), axis=1))

In [6]:
ds = tf.data.Dataset.from_tensor_slices(
    (x_train, tf.one_hot(y_train, 10))
)
ds = ds.batch(100)

In [7]:
for epoch in range(20):
    for x, y in ds:
        loss = train_step(x, y)
    print(epoch, loss.numpy(), evaluate(model))

0 0.0002057658 0.9437
1 0.00021051208 0.9588
2 0.00014657776 0.9715
3 0.000100423626 0.9695
4 9.5032665e-05 0.9706
5 0.000113048525 0.9693
6 0.00013361908 0.9737
7 0.0001144739 0.9748
8 8.687482e-05 0.9732
9 8.595058e-05 0.9733
10 0.00010973251 0.9709
11 9.811082e-05 0.9765
12 8.027111e-05 0.9756
13 8.056414e-05 0.9736
14 0.00013870945 0.9772
15 8.0807826e-05 0.9779
16 9.479944e-05 0.975
17 8.1188686e-05 0.9771
18 9.18084e-05 0.9809
19 8.082365e-05 0.9771


In [8]:
evaluate(model)

0.9771

## Baseline Model with Usual Loss

In [9]:
baseline_model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, 'softmax')
])

In [10]:
baseline_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)

In [11]:
baseline_model.fit(
    x_train, y_train,
    epochs=20,
    validation_data=(x_test, y_test),
    verbose=2,
)

Epoch 1/20
1875/1875 - 2s - loss: 0.2334 - accuracy: 0.9312 - val_loss: 0.1173 - val_accuracy: 0.9633 - 2s/epoch - 1ms/step
Epoch 2/20
1875/1875 - 2s - loss: 0.0973 - accuracy: 0.9699 - val_loss: 0.1054 - val_accuracy: 0.9661 - 2s/epoch - 956us/step
Epoch 3/20
1875/1875 - 2s - loss: 0.0687 - accuracy: 0.9783 - val_loss: 0.0733 - val_accuracy: 0.9779 - 2s/epoch - 962us/step
Epoch 4/20
1875/1875 - 2s - loss: 0.0518 - accuracy: 0.9831 - val_loss: 0.0790 - val_accuracy: 0.9777 - 2s/epoch - 963us/step
Epoch 5/20
1875/1875 - 2s - loss: 0.0402 - accuracy: 0.9870 - val_loss: 0.0842 - val_accuracy: 0.9755 - 2s/epoch - 967us/step
Epoch 6/20
1875/1875 - 2s - loss: 0.0321 - accuracy: 0.9894 - val_loss: 0.0843 - val_accuracy: 0.9773 - 2s/epoch - 966us/step
Epoch 7/20
1875/1875 - 2s - loss: 0.0281 - accuracy: 0.9908 - val_loss: 0.0776 - val_accuracy: 0.9784 - 2s/epoch - 965us/step
Epoch 8/20
1875/1875 - 2s - loss: 0.0246 - accuracy: 0.9920 - val_loss: 0.1008 - val_accuracy: 0.9756 - 2s/epoch - 968us

In [12]:
evaluate(baseline_model)

0.9769

## Model Robustness

Now, we compare the robustness of the model and the baseline. To do so, we add Gaussian noise to the test data and check the accuracy.

In [37]:
stddev = 4e-1
noise = tf.random.normal(tf.shape(x_test)) * stddev

In [38]:
def evaluate_robustness(model):
    accuracy = accuracy_score(
        y_test, tf.argmax(model(x_test), axis=1))
    noised_accuracy = accuracy_score(
        y_test, tf.argmax(model(x_test+noise), axis=1))
    print(f'Accuracy: {accuracy} -> {noised_accuracy}')

In [39]:
evaluate_robustness(model)

Accuracy: 0.9771 -> 0.8849


In [40]:
evaluate_robustness(baseline_model)

Accuracy: 0.9769 -> 0.8078


## Conclusion

By simply using the "gradient loss", we obtain a result that approaches the baseline. But the robustness is greatly out-performs the baseline.